In [6]:
# from flask_other.app import app
from flask_socketio import disconnect as dc
from flask import Flask
from flask_socketio import SocketIO, emit,join_room,leave_room
from flask_socketio import disconnect
from threading import Lock
import os,sys

app=Flask("demo")
from flask import  render_template,request
import time
import re

import pandas as pd
import numpy as np
import time
import re
import requests

import atexit

from apscheduler.schedulers.background import BackgroundScheduler
from apscheduler.triggers.interval import IntervalTrigger



##################################
async_mode = None
app = Flask(__name__)
app.config['SECRET_KEY'] = 'secret!'
socketio = SocketIO(app, async_mode=async_mode)

thread = None
thread_lock = Lock()
clients = []
sentences = []

name_space = '/chat'

@app.route('/')
def index():
    return render_template('index.html', async_mode=socketio.async_mode)


@socketio.on('client_send',namespace=name_space)
def client_msg(msg):  
    uid = request.sid
    sentence = msg.get('data')
    sentence = decode(sentence)
    sentences.append(sentence)
    response = cache.chat(uid, sentence)
    if response == 'end':
        socketio.emit('my_response',{'data':'感谢选择江苏逸能，再见！'},room = uid, namespace=name_space)
        disconnect_frontend(uid)
    elif response is None:
        socketio.emit('my_response',{'data':'当前会话已经过期，感谢选择江苏逸能，再见！'},room = uid, namespace=name_space)
        disconnect_frontend(uid)
    else:
        socketio.emit('my_response',{'data':response},room = uid, namespace=name_space)

def decode(msg):
    msg = re.sub(r'%u', r'\u', msg)
    msg = msg.encode('latin-1').decode('unicode_escape')
    return msg

#sned message to a specific user
def unique_message(uid):
    socketio.emit('unique_messgae',{'data':'are you still here'}, room=uid, namespace=name_space)
    
def disconnect_frontend(uid):
    socketio.emit('my_response',{'status':'disconnected'},room = uid, namespace=name_space)


@socketio.on('disconnect',namespace=name_space)
def disconnect():
    uid = request.sid
    cache.remove_session(uid)
    leave_room(uid)
    dc()
    print('{} is disconnected'.format(uid))
    
@socketio.on('connect',namespace=name_space)
def connect():
    print('connect')
    print(request.sid)
    uid = request.sid
    clients.append(uid)
    print(len(clients))
    
    join_room(uid)
    if cache.create_session(uid):
        response = cache.chat(uid, '')
        if response is not None:
            socketio.emit('my_response',{'data':response},room = uid,namespace=name_space) #the first sentence
            return None
        
    else:
        socketio.emit('my_response',{'data':'server busy. please click new conv'},room = uid,namespace=name_space) 
        disconnect_frontend(uid)
    



In [7]:
print('http://10.0.24.31:8899/')
print('http://0.0.0.0:6006/')
scheduler = BackgroundScheduler()
scheduler.start()
scheduler.add_job(
    func=cache.purge_inactive,
    trigger=IntervalTrigger(seconds=10),
    id='purge_cache',
    name='purge_inactive',
    replace_existing=True)
# Shut down the scheduler when exiting the app
atexit.register(lambda: scheduler.shutdown())

socketio.run(app,'0.0.0.0',6006)

http://10.0.24.31:8899/
http://0.0.0.0:6006/
2f0c2af808b043ecb148de24dc10e79c is disconnected
connect
8ec59734b7fb4208a85d663c13337d79
1


Job "purge_inactive (trigger: interval[0:00:10], next run at: 2018-07-01 16:17:00 UTC)" raised an exception
Traceback (most recent call last):
  File "/home/kai/anaconda3/lib/python3.6/site-packages/apscheduler/executors/base.py", line 125, in run_job
    retval = job.func(*job.args, **job.kwargs)
  File "<ipython-input-5-292ad72f8e9f>", line 58, in purge_inactive
    for uid in self.active_session:
RuntimeError: dictionary changed size during iteration


8ec59734b7fb4208a85d663c13337d79 session is inactive, will be removed!
session 8ec59734b7fb4208a85d663c13337d79 is removed
8ec59734b7fb4208a85d663c13337d79 is disconnected
8ec59734b7fb4208a85d663c13337d79 is disconnected
connect
075df5b55f5243a7b819528d07460cba
2


KeyboardInterrupt
2018-07-01T16:17:45Z


KeyboardInterrupt: 

In [1]:
import time
import gc
class Cache:
    def __init__(self, graph_path,msg_path,model_dict,max_session=1000):
        self.max_session = 1000
        self.inform_interval = 20
        self.inactive_maxlength = 75
        #{'uid': {'stragety': Tree(), 'time_response': <time>, 'time_inform': <>}
        self.active_session = {}
        self.model_dict = model_dict
        self.graph_path = graph_path
        self.msg_path = msg_path
        
        
    def create_session(self, uid):
        if len(self.active_session) < self.max_session:
            self.active_session[uid] = {}
            self.active_session[uid].update({'stragety':TreeStage1(graph_path=self.graph_path,
                                                                   msg_path=self.msg_path)})
            self.active_session[uid].update({'time_response':time.time()})
            self.active_session[uid].update({'time_inform':time.time()})
            self.active_session[uid].update({'chatting':[]})
            return True
        else:
            return False
        
    def remove_session(self,uid):
        response = '您当前的会话超过 {} 秒没有响应，系统将关闭当前会话！如有需求，请开始新的对话！'.format(self.inactive_maxlength)
        try:
            socketio.emit('my_response',{'data':response},room = uid, namespace=name_space)
        except:
            pass
        try:
            disconnect_frontend(uid)
        except:
            pass
        try:
            del self.active_session[uid]
            print('session {} is removed'.format(uid))
        except KeyError:
            pass
        gc.collect()
            
    def chat(self,uid,sentence):
        if self.active_session.get(uid) is not None:
            response = self.active_session[uid]['stragety'].process(sentence, self.model_dict)
            self.active_session[uid]['time_response'] = time.time()
            self.active_session[uid]['time_inform'] = time.time()
            self.active_session[uid]['chatting'].append(response)
        else:
            response = None
        return response
        
    
    
    def purge_inactive(self):
        current = time.time()
        for uid in self.active_session:
            if current - self.active_session[uid]['time_response'] > self.inactive_maxlength:
                print('{} session is inactive, will be removed!'.format(uid))
                self.remove_session(uid)
            try:
                if current - self.active_session[uid]['time_inform'] > self.inform_interval:
                    self.inform_inactive(uid)
            except KeyError:
                pass
                
        
    def inform_inactive(self, uid):
        self.active_session[uid]['time_inform'] = time.time()
        last_sentence = self.active_session[uid]['chatting'][-1]
#         response = '您有在听我说吗？ \n'+last_sentence
        response = '您有在听我说吗?请回答我刚才的问题！'
        socketio.emit('my_response',{'data':response},room = uid, namespace=name_space)
          
################################################
import sys,os
sys.path.append('../../../MLModel/code/OneClickTraining/')
sys.path.append('../../../MLModel/code/TreeModelV2/')
from all_model_py import *
import pickle
from chatbotv1 import *



models_list = ['IDClassifier','CutDebt','IfKnowDebtor','WillingToPay','Installment','ConfirmLoan']
savedModel_path = '../../../MLModel/savedModel/{}/{}.pickle'

model_dict = {}
for each_model in models_list:
    model_dict[each_model] = pickle.load(open(savedModel_path.format(each_model,each_model), 'rb'))
    model_dict[each_model].classify('再说一次')
    
model_dict['StopClassifier'] = StopClassifier()
model_dict['InitClassifier'] = InitClassifier()    

graph_path='../../../MLModel/data/TreeModel/treeConnection.csv'
msg_path='../../../MLModel/data/TreeModel/node_message.csv'
#################################################################
cache = Cache(graph_path=graph_path,
              msg_path=msg_path,
              model_dict=model_dict)

Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.613 seconds.
Prefix dict has been built succesfully.


In [12]:
cache.create_session('1232')

True

In [15]:
cache.active_session['1232']['stragety'].current_node_name

's0'

In [5]:
uid='1232'
cache.chat(uid,'是的')

'您好，您之前借贷了5万块钱，截至今天已经逾期47天，逾期总欠款5万2千块钱，其中利息1500，滞纳金500块。请问为什么到现在都没有处理呢？'

In [11]:
t = TreeStage1(graph_path=graph_path, msg_path=msg_path)

In [17]:
print(t.current_node_name)
print(t.process('我没有钱',model_dict))
print(t.current_node_name)

cf_s1_n1_identity_q
您好，您之前借贷了5万块钱，截至今天已经逾期47天，逾期总欠款5万2千块钱，其中利息1500，滞纳金500块。请问为什么到现在都没有处理呢？
cf_s1_n2_confirmLoan_q


In [18]:
wil = model_dict['WillingToPay']

In [20]:
t.messages

,node_name,label,sentiment,message
0,s0,0,1,你好，这里是江苏逸能催收公司，请问是罗巍先生吗？
1,cf_s1_n1_identity_q,1,1,不好意思，打扰了，请问您认识罗先生吗！
2,cf_s1_n1_identity_q,0,1,您好，您之前借贷了5万块钱，截至今天已经逾期47天，逾期总欠款5万2千块钱，其中利息1500...
3,cf_s1_n1_identity_q,101,1,是这样的，您之前借贷了5万块钱，截至今天已经逾期47天，逾期总欠款5万2千块钱，其中利息15...
4,cf_s1_n1_identity_q,103,1,我们这里是江苏逸能催收公司的，请问您是罗巍先生吗？
5,cf_s1_n1_identity_q,104,1,那好吧，那我过一会儿再给您打过去。再见。
6,cf_s1_n1_identity_q,107,1,我这边是江苏逸能催收公司的催收员。您之前借贷了5万块钱，截至今天已经逾期47天，逾期总欠款5...
7,cf_s1_n1_identity_q,109,1,请你不要说这些有的没的，您是罗巍先生吗？
8,cf_s1_n1_identity_q,110,1,你说这些乱七八糟的都没有用的，这个该还的你肯定是跑不掉的。你现在告诉我你是罗巍先生吗？
9,cf_s1_n2_confirmLoan_q,0,1,您之前借贷了5万块钱，截至今天已经逾期47天，逾期总欠款5万2千块钱，其中利息1500，滞纳...


In [ ]:
wil.classify 